# Summarizing the Transaction Data 

In [381]:
import pandas as pd
import numpy as np

In [382]:
full_dataset = pd.read_csv("bills.csv") # use location of bills dataset

We are hoping to summarize data from the "transaction" level to the "day" level. The goal is to get a record per day for each recorded restaurant. 

In [ ]:
# take a look at a few rows
full_dataset.head(5)

In [384]:
full_dataset.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8954786 entries, 0 to 8954785
Data columns (total 20 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   bill_paid_at_local              object 
 1   bill_total_billed               float64
 2   bill_total_discount_item_level  float64
 3   bill_total_gratuity             float64
 4   bill_total_net                  float64
 5   bill_total_tax                  float64
 6   bill_total_voided               float64
 7   bill_uuid                       object 
 8   business_date                   object 
 9   order_duration_seconds          int64  
 10  order_seated_at_local           object 
 11  order_closed_at_local           object 
 12  order_take_out_type_label       object 
 13  order_uuid                      object 
 14  payment_amount                  float64
 15  payment_count                   int64  
 16  payment_total_tip               float64
 17  sales_revenue_with_tax     

In order to summarize by day, subtotals or averages need to be created.  
**Subtotals:**   
 1   bill_total_billed               float64  
 2   bill_total_discount_item_level  float64  
 3   bill_total_gratuity             float64  
 4   bill_total_net                  float64  
 5   bill_total_tax                  float64  
 6   bill_total_voided               float64  
 14  payment_amount                  float64  
 15  payment_count                   int64  
 16  payment_total_tip               float64  
 17  sales_revenue_with_tax          float64  
**Average:**   
9   order_duration_seconds          int64  
**Counts:**  
12  order_take_out_type_label       object  
19  waiter_uuid                     object (distinct)   
**Keep/Identifiers:**   
8   business_date                   object  
18  venue_xref_id                   object  

In [385]:
# Let's see how many unique venues are the in dataset 
full_dataset['venue_xref_id'].nunique()

601

In [386]:
# check number of null values 
full_dataset.isnull().sum()

bill_paid_at_local                 0
bill_total_billed                  0
bill_total_discount_item_level     0
bill_total_gratuity                0
bill_total_net                     0
bill_total_tax                     0
bill_total_voided                  0
bill_uuid                          0
business_date                      0
order_duration_seconds             0
order_seated_at_local              0
order_closed_at_local              0
order_take_out_type_label          0
order_uuid                         0
payment_amount                     0
payment_count                      0
payment_total_tip                  0
sales_revenue_with_tax             0
venue_xref_id                      0
waiter_uuid                       94
dtype: int64

We want to add the counts of the different order types ('takeout', 'dine-in', etc). Let's add flags for the different types of orders.

In [387]:
# how many order types are we expecting? and what are they? 
order_types = full_dataset['order_take_out_type_label'].unique()
order_types

array(['dinein', 'bartab', 'takeout', 'delivery', 'onlineorder'],
      dtype=object)

In [ ]:
# add flag columns 
flagged_data = full_dataset 
for label in order_types: 
    flagged_data[label] = np.where(flagged_data['order_take_out_type_label'] == label, 1, 0)
flagged_data[['order_take_out_type_label', 'dinein', 'bartab', 'takeout', 'delivery', 'onlineorder']]

To get the data into rows by day instead of transaction, let's group by restaurant and date as the identifiers. 

In [ ]:
grouped_dataset = flagged_data.groupby(['venue_xref_id', 'business_date']).agg(
    {'bill_total_billed' : 'sum', 
     'bill_total_discount_item_level' : 'sum',
     'bill_total_gratuity' : 'sum',
     'bill_total_net' : 'sum',
     'bill_total_tax' : 'sum',
     'bill_total_voided' : 'sum',
     'payment_amount' : 'sum', 
     'payment_count' : 'sum', 
     'payment_total_tip' : 'sum', 
     'sales_revenue_with_tax' : 'sum', 
     'order_duration_seconds' : 'mean',
     'dinein' : 'sum', 
     'bartab' : 'sum',
     'takeout' : 'sum', 
     'delivery' : 'sum',
     'onlineorder' : 'sum',
     'waiter_uuid' : 'nunique'
    }).reset_index()

In [ ]:
# check the totals of the grouped vs the original 
full_sum = full_dataset[['bill_total_billed', 'bill_total_discount_item_level', 'bill_total_gratuity', 'bill_total_net', 'bill_total_tax', 'bill_total_voided', 'payment_amount', 'payment_count', 'payment_total_tip', 'sales_revenue_with_tax']].sum()
group_sum = grouped_dataset[['bill_total_billed', 'bill_total_discount_item_level', 'bill_total_gratuity', 'bill_total_net', 'bill_total_tax', 'bill_total_voided', 'payment_amount', 'payment_count', 'payment_total_tip', 'sales_revenue_with_tax']].sum()
print(full_sum)
print(group_sum)
print(full_sum.apply(int) == group_sum.apply(int))


In [391]:
# Export the data
grouped_dataset.to_csv('dailyTransactions.csv', index=False) 